In [1]:
import pandas as pd
import numpy as np
import tqdm
import warnings
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score

pd.set_option("max_rows", 200)
pd.set_option("max_columns", 50)
tqdm.tqdm.pandas()
%matplotlib inline

In [2]:
df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle").sort_values(["user_id", "timestamp"]).reset_index(drop=True)

In [3]:
import sys
sys.path.append("../")
# df = pd.read_pickle("../input/riiid-test-answer-prediction/split10/train_0.pickle")
df = df.sort_values(["user_id", "timestamp"])
# large_user_id = df["user_id"].value_counts()
# large_user_id = large_user_id[large_user_id > 1000] 
# df = df[df["user_id"].isin(large_user_id.index)]
# df = df[df["answered_correctly"] != -1]

In [4]:
df["answered_correctly"] = df["answered_correctly"].replace(-1, np.nan)

In [5]:
def f(series):
    return series.shift(1).cumsum().fillna(0) / np.arange(len(series))
df["target_enc_user_id"] = df.groupby("user_id")["answered_correctly"].transform(f)

In [24]:
df = df[df["content_type_id"] == 0]
df["reverse_prior_question_had_explanation"] = df["prior_question_had_explanation"] == 0

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
df["session0_sum"] = df.groupby("user_id")["reverse_prior_question_had_explanation"].transform(sum)

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [29]:
df[df["session0_sum"] == 2101].to_csv("aaa.csv")

In [27]:
df["session0_sum"].describe()

count    9.837542e+06
mean     6.565752e+01
std      1.296482e+02
min      0.000000e+00
25%      1.100000e+01
50%      3.000000e+01
75%      4.900000e+01
max      2.101000e+03
Name: session0_sum, dtype: float64

In [28]:
df.groupby("session0_sum")["answered_correctly"].mean()

session0_sum
0       0.645284
1       0.675314
2       0.582415
3       0.701480
4       0.682572
          ...   
986     0.797629
1054    0.763512
1072    0.689636
1814    0.806430
2101    0.678630
Name: answered_correctly, Length: 364, dtype: float64

In [31]:
df["timestampover1e7"] = df["timestamp"] > 1e+7

/home/owner/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
df.groupby(["timestampover1e7", "prior_question_had_explanation"])["answered_correctly"].describe()

count      mean  \
timestampover1e7 prior_question_had_explanation                        
False            False                            627870.0  0.450722   
                 True                             331542.0  0.640875   
True             False                            282602.0  0.604384   
                 True                            8556344.0  0.672105   

                                                      std  min  25%  50%  75%  \
timestampover1e7 prior_question_had_explanation                                 
False            False                           0.497566  0.0  0.0  0.0  1.0   
                 True                            0.479745  0.0  0.0  1.0  1.0   
True             False                           0.488984  0.0  0.0  1.0  1.0   
                 True                            0.469447  0.0  0.0  1.0  1.0   

                                                 max  
timestampover1e7 prior_question_had_explanation       
False            False                           1.0  
                 True                            1.0  
True             False                           1.0  
                 True                            1.0

In [ ]:
df["timestampover1e6"] = df["timestamp"] > 1e+8

In [ ]:
df.groupby(["timestampover1e6", "prior_question_had_explanation"])["answered_correctly"].describe()